# MNIST convolutional neural networks

* MNIST data를 가지고 **convolutional neural networks**를 만들어보자.
* reference codes
  * [tensorflow/tutorials/mnist_deep.py](https://github.com/tensorflow/tensorflow/blob/r1.12/tensorflow/examples/tutorials/mnist/mnist_deep.py)
  * [cs20/examples/07_convnet_mnist.py](https://github.com/chiphuyen/stanford-tensorflow-tutorials/blob/master/examples/07_convnet_mnist.py)

## Import modules

In [ ]:
"""A very simple MNIST classifier.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

import tensorflow as tf

slim = tf.contrib.slim

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
os.environ["CUDA_VISIBLE_DEVICES"]="0"

## Import data

In [ ]:
# Load training and eval data from tf.keras
(train_data, train_labels), (test_data, test_labels) = \
    tf.keras.datasets.mnist.load_data()

train_data = train_data / 255.
train_labels = np.asarray(train_labels, dtype=np.int32)

test_data = test_data / 255.
test_labels = np.asarray(test_labels, dtype=np.int32)

### Show the MNIST

In [ ]:
index = 100
print("label = {}".format(train_labels[index]))
plt.imshow(train_data[index])
plt.colorbar()
plt.gca().grid(False)
plt.show()

## Set up dataset with `tf.data`

### create input pipeline with `tf.data.Dataset`

In [ ]:
tf.set_random_seed(219)
batch_size = 32
max_epochs = 1

# for train
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size = 10000)
train_dataset = train_dataset.repeat(count = max_epochs)
train_dataset = train_dataset.batch(batch_size = batch_size)
print(train_dataset)

# for test
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))
test_dataset = test_dataset.batch(batch_size = len(test_data))
print(test_dataset)

### Define Iterator

In [ ]:
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(handle,
                                               train_dataset.output_types,
                                               train_dataset.output_shapes)
images, labels = iterator.get_next()
images = tf.cast(images, dtype = tf.float32)
labels = tf.cast(labels, dtype = tf.int32)

## Build the model

In [ ]:
class CnnMNIST(object):
  def __init__(self, inputs, labels):
    self.inputs = inputs
    self.labels = labels
    self.num_classes = 10
    
  def inference(self, inputs):
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel
    # TODO
    self.inputs_images = 

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    # TODO
    conv1 = 

    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    # TODO
    pool1 = 

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    # TODO
    conv2 = 

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    # TODO
    pool2 = 

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = 

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    # TODO
    dense = 

    # Add dropout operation; 0.6 probability that element will be kept
    self.is_training = tf.placeholder(tf.bool)
    # TODO
    dropout = 

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    # TODO
    logits = 

    return logits
  
  def cross_entropy_loss(self, labels, logits):
    #y_one_hot = tf.one_hot(y, depth=10)
    #cross_entropy = tf.losses.softmax_cross_entropy(onehot_labels=y_one_hot, logits=y_pred)
    cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits, scope='loss')
    
    return cross_entropy
  
  def predictions(self):
    with tf.variable_scope('predictions'):
      predictions = tf.argmax(self.logits, 1)
      
      return predictions
      
  
  def build(self):
    self.global_step = tf.train.get_or_create_global_step()
    
    self.logits = self.inference(self.inputs)
    self.loss = self.cross_entropy_loss(self.labels, self.logits)
    self.predictions = self.predictions()
    
    print("complete model build.")

### Build the model

In [ ]:
model = CnnMNIST(images, labels)
model.build()

# show info for trainable variables
t_vars = tf.trainable_variables()
slim.model_analyzer.analyze_vars(t_vars, print_info=True)

## Define optimizer

In [ ]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(model.loss, global_step=model.global_step)

### Assign `tf.summary.FileWriter`

In [ ]:
graph_location = 'graphs/01.cnn.mnist.tf.data'
print('Saving graph to: %s' % graph_location)
train_writer = tf.summary.FileWriter(graph_location)
train_writer.add_graph(tf.get_default_graph())

### `tf.summary`

In [ ]:
with tf.name_scope('summaries'):
  tf.summary.scalar('loss/cross_entropy', model.loss)
  tf.summary.image('images', model.inputs_images)
  for var in tf.trainable_variables():
    tf.summary.histogram(var.op.name, var)
  # merge all summaries
  summary_op = tf.summary.merge_all()

## `tf.Session()`

In [ ]:
print("Session start!")
start_time = time.time()
sess = tf.Session(config=sess_config)
sess.run(tf.global_variables_initializer())

# train_iterator
train_iterator = train_dataset.make_one_shot_iterator()
train_handle = sess.run(train_iterator.string_handle())
duration = time.time() - start_time
print("elapsed time making train data: {} sec".format(duration))

## Training

In [ ]:
print("training start!")
while True:
  try:
    start_time = time.time()
    _, global_step_, loss = sess.run([train_step, model.global_step, model.loss],
                                     feed_dict={handle: train_handle,
                                                model.is_training: True})
    if global_step_ % 10 == 0:
      clear_output(wait=True)
      duration = time.time() - start_time
      examples_per_sec = batch_size / float(duration)
      epochs = batch_size * global_step_ / float(len(train_data))
      print("Epochs: {:.2f} global_step: {} loss: {:.3f} ({:.2f} examples/sec; {:.3f} sec/batch)".format(
                epochs, global_step_, loss, examples_per_sec, duration))

    if global_step_ % 200 == 0:
      # summary
      summary_str = sess.run(summary_op, feed_dict={handle: train_handle, model.is_training: False})
      train_writer.add_summary(summary_str, global_step=global_step_)

    #if global_step_ > 100:
    #  break

  except tf.errors.OutOfRangeError:
    print("End of dataset")  # ==> "End of dataset"
    break

train_writer.close()
print("training done!")

## Test trained model

* test accuracy: 0.9804 for 1 epoch

In [ ]:
# test_iterator
test_iterator = test_dataset.make_one_shot_iterator()
test_handle = sess.run(test_iterator.string_handle())

### Use `tf.metrics`

In [ ]:
accuracy, acc_op = tf.metrics.accuracy(labels=labels,
                                       predictions=model.predictions, name='accuracy')
sess.run(tf.local_variables_initializer())

sess.run(acc_op, feed_dict={handle: test_handle, model.is_training: False})
print("test accuracy:", sess.run(accuracy))

### Plot test set

In [ ]:
np.random.seed(219)

In [ ]:
test_batch_size = 16
batch_index = np.random.choice(len(test_data), size=test_batch_size, replace=False)

batch_xs = test_data[batch_index]
batch_ys = test_labels[batch_index]
y_pred = sess.run(model.predictions, feed_dict={images: batch_xs, model.is_training: False})

fig = plt.figure(figsize=(16, 10))
for i, (px, py) in enumerate(zip(batch_xs, y_pred)):
  p = fig.add_subplot(4, 8, i+1)
  if py == batch_ys[i]:
    p.set_title("y_pred: {}".format(py), color='blue')
  else:
    p.set_title("y_pred: {}".format(py), color='red')
  p.imshow(px.reshape(28, 28))
  p.axis('off')